### M&uuml;hle-Koordinaten: 

Man kann die Position eines Steins auf dem
M&uuml;hlebrett durch `(r, i)` beschreiben, wobei
- `r` ist einer der Ringe 0, 1, 2 (von innen nach aussen)
- `i` ist einer der Punkte 8 Punkte auf diesem Ring, nummeriert z.B.  
$\begin{array}{ccc}
6& 5& 4\\
7 &&  3\\
0 &1 &2\\
\end{array}$  

Wir wollen diese 3 Ringe zeichnen und darauf die Punkte markieren, wo Steine gesetzt werden k&ouml;nnen. 
Klickt man auf einen Punkt, so wird dort ein Stein gesetzt und
ausgegeben, welche M&uuml;hle-Koordinate dieser Punkt hat.

In [1]:
RADIUS = 8
REL_SIZE = 0.8
SIZE = 200

# Punkte eines Rings mit Zentrum (0,0)
PTS = [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), 
       (0, 1), (-1, 1), (-1, 0),
      ]

def ri2xy(r, i):
    x, y = PTS[i]
    x = (REL_SIZE*(r+1)/3*x + 1) * SIZE/2
    y = (-REL_SIZE*(r+1)/3*y + 1) * SIZE/2
    return x, y

def draw_board(canvas):
    for r in range(3):
        canvas.stroke_polygon([ri2xy(r, i) for i in range(8)])
        for i in range(8):
            canvas.fill_circle(*ri2xy(r, i), 3)
    
    for i in (1,3,5,7):
        bg.stroke_lines([ri2xy(r, i) for r in range(3)])

In [4]:
from get_closest import get_closest
from ipywidgets import Output
from ipycanvas import MultiCanvas

out = Output(layout = {'border': '1px solid black'})
mcanvas = MultiCanvas(2, width = SIZE, height = SIZE, layout = {'border': '1px solid black'})
bg, fg = mcanvas
fg.fill_style = 'blue'

pts = [ri2xy(r, i) for r in range(3) for i in range(8)]
idx_pt = [(r, i) for r in range(3) for i in range(8)]

@out.capture(clear_output=True)        
def on_mouse_down(x, y):
    idx = get_closest(pts, (x, y))
    if idx is not None:
        fg.fill_circle(*pts[idx], RADIUS)
        pt = idx_pt[idx]
        print('Stein auf Position {}'.format(pt))
  
        
mcanvas.on_mouse_down(on_mouse_down)

display(mcanvas, out)

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [5]:
draw_board(bg)

In [6]:
fg.clear()
out.clear_output()

### M&uuml;hle, Skizze
Die Spielposition ist z.B. eine Tabelle mit 3 Zielen und 9 Spalten und Eintr&auml;gen `X`, `O` oder `None`.
```python
[[None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
]
```

Man kann von `(r,i)` nach  `(s,j)` ziehen, falls
- `r == s and abs(i - j) in (1, 7)` (benachbart im gleichen Ring)
- `abs(r - s) == 1 and i == j and i % 2 == 1` (benachbart auf Verbindung zw. Ringen)

Auf M&uuml;hle testen:  
Alle m&ouml;glichen M&uuml;hlen (Liste von 3 Feldern in einer Reihe)
als Liste abspeichern.
```python
muehlen = [[(0, 0), (0, 1), (0, 2)],
           [(0, 2), (0, 3), (0, 4)],
           ...
          ] 

def is_muehle(pos):
    '''gibt True zurueck, falls der Stein an Position pos 
       zu einer Muehle gehoert
    '''
    # finde die Muehle, die (r, i) enthaelt
    # teste, ob alle Steine in dieser Muehle dem gleichen Spieler gehoeren
```
Ev. weitere n&uuml;tzliche Funktionen:
```python

def cannot_move(player):
    '''teste, ob ein Spieler eingeschlossen ist,
       ob alle Steine des Spielers blockiert sind
    '''
    pass

def is_blocked(pos):
    ''' testet, ob der Stein an Position pos blockiert ist'''
    neighbors = get_neighbors(pos)
    # teste ob alle Felder in neighbors besetzt sind
    
def get_neighbors(pos):
    '''Finde alle Nachbarn der Position pos'''
    pass

def are_are_neighbors(pos1, pos2):
    '''testet, ob man von pos1 nach pos2 ziehen kann'''
    pass
```    

### Grobe Skizze einer Game Klasse

In [43]:
class Game:
    def __init__(self):
        self.players = {0: 'X', 1: 'O'}
        self.position = [[None] * 9 for _ in range(3)]
        self.ptm = 0
        self.winner = None
        
    def is_legal(self, r, i, player):
        if player != self.ptm:
            return False
        
        return self.position[r][i] is None
        
    def move(self, r, i, player):
        if not self.is_legal(r, i, player):
            return
        
        self.position[r][i] = self.players[player]
        
        # teste ob Muehle, falls, ja bleibt Spieler am Zug und 
        # als weiterer Zug einen Stein des Gegners entfernen
        
        # teste ob ein Spieler gewonnen hat und das Spiel endet
        if self.is_winner(player):
            self.winner = player
            self.ptm = None
            return
        
        self.ptm = 1 - self.ptm   
        
    def is_winner(self, player):
        '''teste ob Gegener nicht ziehen kann oder,
           falls Setzphase abgeschlossen, weniger als 3 Steine hat
        '''   
        pass
    
    def position2str(self):
        rows = [None for _ in range(7)]
        rings = [''.join(char or '.' for char in ring) for ring in self.position]
        ring0, ring1, ring2 = rings

        rows[0] = '{}  {}  {}'.format(*ring2[6:3:-1])
        rows[1] = ' {} {} {} '.format(*ring1[6:3:-1])
        rows[2] = '  {}{}{}  '.format(*ring0[6:3:-1])
        
        rows[3] = '{}{}{} {}{}{}'.format(ring2[7], ring1[7], ring0[7], 
                                         ring0[3], ring1[3], ring2[3]
                                        )
        
        rows[4] = '  {}{}{}  '.format(*ring0[:3])
        rows[5] = ' {} {} {} '.format(*ring1[:3])
        rows[6] = '{}  {}  {}'.format(*ring2[:3])
    
        return '\n'.join(rows)   
    
    def __repr__(self):
        return 'Am Zug: {}\nWinner: {}\n{}'.\
                format(self.players.get(self.ptm),
                       self.players.get(self.winner),
                       self.position2str(),
                      )

In [46]:
game = Game()
game

Am Zug: X
Winner: None
.  .  .
 . . . 
  ...  
... ...
  ...  
 . . . 
.  .  .

In [48]:
game.move(2, 0, 0)
game

Am Zug: O
Winner: None
.  .  X
 . O . 
  ...  
... .O.
  ...  
 . O . 
X  X  X

In [47]:
game = Game()

moves = [(2, 0), (1, 1), (2, 4), (1, 3), (2, 2), (1, 5), (2, 1)]
for j, (r, i) in enumerate(moves):
    player = j % 2
    game.move(r, i, player)
game

Am Zug: O
Winner: None
.  .  X
 . O . 
  ...  
... .O.
  ...  
 . O . 
X  X  X